In [ ]:
from bottle import Bottle, route, run, template, get, post, debug, static_file, request, redirect
from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
from datetime import date
import datetime as dt
import pandas as pd
import json
import pprint
import sqlite3



def query (query):
    conn = sqlite3.connect('CryptoDB.db')
    c = conn.cursor()
    c.execute(query)
    result = c.fetchall()
    conn.close()
    return result

def convert_dict(tupl, dic):
    for a, b in tupl:
        dic.setdefault(a,[]).append(b)
    return dic


app = Bottle()

@app.get(r'/static/css/<filepath:re:.*\.css>')
def css(filepath):
    return static_file(filepath, root="static/css")

@app.get(r'/static/img/<filepath:re:.*\.(jpg|png|gif|ico|svg)>')
def img(filepath):
    return static_file(filepath, root="static/img")

@app.get(r'/static/js/<filepath:re:.*\.js>')
def js(filepath):
    return static_file(filepath, root="static/js")



@app.route('/')
@app.route('/index')
@app.route('/index/')
def base():
    return template('home')

@app.route('/home')
def home():
    return template('home')

@app.get('/login')
@app.get('/login/')
def login():
    return template('login')

@app.post('/login')
@app.post('/login/')
def login_rec(): 
    
    
    db_users = query('Select username,password from Users order by user_id')
    dictionary = {}
    db_users_dict = convert_dict(db_users, dictionary)

    
    got_login = request.forms.get('login', default=False)
    got_password = request.forms.get('password', default=False)
    
    if (got_login in db_users_dict) and db_users_dict[got_login][0] == got_password:
        redirect('/index')
        return True
    return template('login')

@app.get('/createAccount')
def register_get():
  
   return template('createAccount')


@app.post('/createAccount')
def register_post():   
    
    conn = sqlite3.connect('CryptoDB.db')
    c = conn.cursor()
    c.execute('Select  username from Users')
    logins = c.fetchall()
    c.execute('Select  email from Users')
    emails = c.fetchall()
    
    nlogins =[]
    nemails = []
    for login in logins:
        nlogins.append(login[0])
    for email in emails:
        nemails.append(email[0])
        
        
    login = request.forms.get('login')
    password = request.forms.get('password')
    email = request.forms.get('email')
    
    if(login in nlogins or email in nemails):
        return redirect('/createAccount')
    else: 
        c.execute("Insert into USERS values(null,?,?,?,?)",(str(login),str(password),str(email),date.today()))
        conn.commit()
        conn.close() 
        return redirect('/login/')
    
    
    
@app.route('/prices')
def prices():
    conn = sqlite3.connect('CryptoDB.db')

    c = conn.cursor()
    
    c.execute('SELECT C.NAME, A.PRICE, A.VOLUME_24H, A.MARKET_CAP FROM ACTUAL_DATA A LEFT JOIN CRYPTO_DICT C ON C.CRYPT_SYMBOL = A.CRYPTO_SYMBOL')
    table1 = c.fetchall()
    df = pd.DataFrame(table1, columns = ['name', 'price', 'volume', 'market_cap']).to_html(justify = "center", classes = "table table-dark table-hover", index = True)
    
    conn.close() 
    return template('prices', data = df)

@app.get('/graph')
@app.get('/graph/')
def choice(): 
    conn = sqlite3.connect('CryptoDB.db')

    c = conn.cursor()
    c.execute('select distinct  c.name from historical_data h left join crypto_dict c on c.crypt_symbol = h.crypto_symbol where crypto_symbol != "nan" ')
    choice_list = c.fetchall()
    df_choice = pd.DataFrame(choice_list, columns = ['Name'])
    cryptos = df_choice['Name'].tolist()
    
    conn.close() 
    return template('list', title = 'crypto', cryptos = cryptos)
    

@app.post('/graph')
def graph():
    
    chosen = request.forms.get('chosen')
    
    conn = sqlite3.connect('CryptoDB.db')
    c = conn.cursor()
    
    c.execute('select h.date, h.close from historical_data h left join crypto_dict c on c.crypt_symbol = h.crypto_symbol  where c.name = ?', (str(chosen),))
    chart = c.fetchall()
    df = pd.DataFrame(chart, columns = ['Date', 'price'])
    day = df['Date'].tolist()
    price = df['price'].tolist()
    conn.close() 
    return template('graph', day = day, price = price, title = str(chosen))
    
run(app, host = 'localhost', port = 8080, debug = True)

Bottle v0.12.19 server starting up (using WSGIRefServer())...
Listening on http://localhost:8080/
Hit Ctrl-C to quit.

127.0.0.1 - - [27/Dec/2021 10:17:32] "POST /login HTTP/1.1" 303 0
127.0.0.1 - - [27/Dec/2021 10:17:32] "GET /index HTTP/1.1" 200 3800
127.0.0.1 - - [27/Dec/2021 10:17:34] "GET /createAccount HTTP/1.1" 200 2525
127.0.0.1 - - [27/Dec/2021 10:17:44] "POST /createAccount HTTP/1.1" 303 0
127.0.0.1 - - [27/Dec/2021 10:17:44] "GET /login/ HTTP/1.1" 200 2439
127.0.0.1 - - [27/Dec/2021 10:17:45] "GET /login/static/css/login.css HTTP/1.1" 404 772
127.0.0.1 - - [27/Dec/2021 10:17:45] "GET /login/static/css/login.css HTTP/1.1" 404 772
127.0.0.1 - - [27/Dec/2021 10:17:49] "POST /login HTTP/1.1" 303 0
127.0.0.1 - - [27/Dec/2021 10:17:49] "GET /index HTTP/1.1" 200 3800
127.0.0.1 - - [27/Dec/2021 10:17:56] "GET /login HTTP/1.1" 200 2439
127.0.0.1 - - [27/Dec/2021 10:18:01] "POST /login HTTP/1.1" 200 2439
127.0.0.1 - - [27/Dec/2021 10:18:02] "POST /login HTTP/1.1" 200 2439
127.0.0.1 - 

In [15]:
def convert_dict(tupl, dic):
    for a, b in tupl:
        dic.setdefault(a,[]).append(b)
    return dic
      
# Driver Code    







AttributeError: 'NoneType' object has no attribute 'append'

In [46]:
#commit Jarek 22.12
#Poprawilem api, trzeba po innym linku sie laczyc aby seryjnie zaciagac 

from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json
import pprint

import pandas as pd

url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest'

currency = 'USD'

parameters = {
  'convert': currency
}
headers = {
  'Accepts': 'application/json',
  'X-CMC_PRO_API_KEY': '25bdeaa9-d947-4b43-83b5-f2ee7bb97197',
}

session = Session()
session.headers.update(headers)
coins_names = []
coins_prices = []
coins_volume_24h = []
coins_market_cap = []
coins_shortcut = []
coins_id = []
coins_mining = []
coins_total_supply = []
coins_max_supply = []


try:
    response = session.get(url, params=parameters)
    
except (ConnectionError, Timeout, TooManyRedirects) as e:
    print(e)
    
for coins in range(100):
    coins_names.append((json.loads(response.text)['data'][coins]['name']))
    coins_prices.append((json.loads(response.text)['data'][coins]['quote'][currency]['price']))
    coins_volume_24h.append((json.loads(response.text)['data'][coins]['quote'][currency]['volume_24h']))
    coins_market_cap.append((json.loads(response.text)['data'][coins]['quote'][currency]['market_cap']))
    coins_shortcut.append((json.loads(response.text)['data'][coins]['symbol']))
    coins_total_supply.append((json.loads(response.text)['data'][coins]['total_supply']))
    coins_max_supply.append((json.loads(response.text)['data'][coins]['max_supply']))
    try:
        if (json.loads(response.text)['data'][coins]['tags'][0] == 'mineable'):
            coins_mining.append((json.loads(response.text)['data'][coins]['tags'][0]))
        else:
            coins_mining.append('not-mineable')
    except: 
        coins_mining.append('not-mineable')
        
rtime = json.loads(response.text)['data'][1]["last_updated"]
data_act = list(zip(coins_shortcut,coins_prices, coins_volume_24h, coins_market_cap))
data_dic = list(zip(coins_shortcut, coins_names, coins_mining, coins_total_supply, coins_max_supply))
actuals_db = pd.DataFrame(data_act, columns = ['Crypto', 'Price', 'volume 24h', 'Market Cap'])
crypto_dict_db = pd.DataFrame(data_dic, columns = ['crypto_symbol', 'name', 'mining', 'supply', 'max_supply'])

#print("Update time:", rtime)
#print(coins_shortcut)
print(crypto_dict_db.head(100))

conn = sqlite3.connect('CryptoDB.db')

c = conn.cursor()

#insert new student Denis Rodman
#print("Insert new student Denis Rodman into student table\n")

c.execute("Delete from ACTUAL_DATA")
for index, row  in actuals_db.iterrows():
    c.execute("Insert into ACTUAL_DATA values(?,?,?,?)",(str(row['Crypto']),str(row['Price']),str(row['volume 24h']),str(row['Market Cap'])))

c.execute("Delete from CRYPTO_DICT")

for index, row  in crypto_dict_db.iterrows():
    c.execute("Insert into CRYPTO_DICT values(?,?,?,?,?)",(str(row['crypto_symbol']),str(row['name']),str(row['mining']),str(row['supply']),str(row['max_supply'])))


conn.commit()


conn.close()    



   crypto_symbol          name        mining        supply   max_supply
0            BTC       Bitcoin      mineable  1.891158e+07   21000000.0
1            ETH      Ethereum      mineable  1.189002e+08          NaN
2            BNB  Binance Coin  not-mineable  1.668011e+08  166801148.0
3           USDT        Tether  not-mineable  8.075300e+10          NaN
4            SOL        Solana  not-mineable  5.116169e+08          NaN
..           ...           ...           ...           ...          ...
95         AUDIO        Audius  not-mineable  1.066859e+09          NaN
96           OMG   OMG Network  not-mineable  1.402454e+08  140245399.0
97          USDP    Pax Dollar  not-mineable  9.456429e+08          NaN
98          BORA          BORA  not-mineable  1.205750e+09          NaN
99          WAXP           WAX  not-mineable  3.826597e+09          NaN

[100 rows x 5 columns]


In [4]:
#commit Jarek 23.11 
#Dodane API z yahoo do zaciagania danych historycznych 
#dla kazdego symbolu krypto zaciagnietego z CMC pobiera dane historyczne z yahoo, ktore zapisuje do listy dfów 'historical'
#jak wpadna nowe rekordy (bedzie nowy dzien) pobierze tylko te notowania, ktorych mu brakuje i zconcatuje do aktualnego df 
#mozna wybrac walute, ale yahoo nie obsluguje PLN -> USD, EUR smiga 
 
import pandas_datareader as pdr
from datetime import date
import datetime as dt
import json 
import requests


today = date.today().strftime("%Y,%m,%d")
parts = today.split(",")
year_e, month_e, day_e = int(parts[0]), int(parts[1]), int(parts[2])
historical = []
ac_historical = []
x = 0


for coin in coins_shortcut:
    
    try:
    
        y = historical[x].sort_values(by = 'Date', ascending = True).tail(1)
        year_s, month_s, day_s  = (y['Date']).dt.year.values[0], (y['Date']).dt.month.values[0], (y['Date']).dt.day.values[0]
        start = dt.datetime(year_s,month_s,day_s)
        end = dt.datetime(year_e,month_e,day_e)
        df = ac_historical
        
    except:
        
        start = dt.datetime(2000,1,1)
        end = dt.datetime(year_e,month_e,day_e)
        df = historical
        
    try:
        tag = coin + '-' + currency
        df.append(pdr.get_data_yahoo(tag, start , end ))
        df[x]['Crypto'] = coin
        df[x].reset_index(inplace = True, drop = False)
        
    except:

        try:
            tag = coin + '1-' + currency
            df.append(pdr.get_data_yahoo(tag, start , end ))
            df[x]['Crypto'] = coin
            df[x].reset_index(inplace = True, drop = False)
        except:

            try:
                tag = coin + '3-' + currency
                df.append(pdr.get_data_yahoo(tag, start , end ))
                df[x]['Crypto'] = coin
                df[x].reset_index(inplace = True, drop = False)
            except:
                pass
    x += 1
    


historical_df = pd.concat(historical)

try:
    ac_historical_df = pd.concat(ac_historical)
    historical = [historical_df, ac_historical_df]
    historical_base = pd.concat(historical)
    historical_base.reset_index(inplace = True, drop = True)
    
except: 
    historical_base = historical_df
    historical_base.reset_index(inplace = True, drop = True)

historical_base.tail(20)
   



,Date,High,Low,Open,Close,Volume,Adj Close,Crypto
96504,2021-12-07,0.510557,0.472023,0.486144,0.485799,84482661.0,0.485799,WAXP
96505,2021-12-08,0.728246,0.456406,0.485070,0.663711,846794078.0,0.663711,WAXP
96506,2021-12-09,0.711925,0.562528,0.673510,0.563387,606791135.0,0.563387,WAXP
96507,2021-12-10,0.587127,0.505400,0.565957,0.521420,279236803.0,0.521420,WAXP
96508,2021-12-11,0.540171,0.502898,0.524753,0.535177,105289850.0,0.535177,WAXP
96509,2021-12-12,0.535306,0.508792,0.535306,0.527960,61390324.0,0.527960,WAXP
96510,2021-12-13,0.531475,0.437049,0.528968,0.446578,92485198.0,0.446578,WAXP
96511,2021-12-14,0.448329,0.424490,0.445093,0.424490,78624869.0,0.424490,WAXP
96512,2021-12-15,0.469647,0.417154,0.447339,0.465307,75088703.0,0.465307,WAXP
96513,2021-12-16,0.493715,0.453991,0.465600,0.454060,74353500.0,0.454060,WAXP
